In [7]:
import pandas as pd
import numpy as np

In [8]:
DATASET = "../data/cs-training.csv"
COLUMN_TARGET = "SeriousDlqin2yrs"

df = pd.read_csv(DATASET)
df.head()

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [ ]:


def main():
    df = pd.read_csv(DATASET)

# Remover coluna lixo
    df = df.drop(columns=["Unnamed: 0"])

# Removação de idade válida (Identifica no EDA 01)
    df = df[df["age"] > 0].copy()

# Essas colunas apresentaram grande quantidade nulos no EDA, porém vou escolher não deletar essas linhas pois representa 20% das linhas.
# Então irei criar uma coluna com uma flag indicando que estava faltando
# Os valores nulos serão preenchidos com a mediana, se usar a média pode ser afetado por outliers
    df["MonthlyIncome_missing"] = df["MonthlyIncome"].isna().astype(int) # caso seja null, retorna True
    df["Dependents_missing"] = df["NumberOfDependents"].isna().astype(int) # caso seja null, retorna True

# Os valores nulos serão preenchidos com a mediana, se usar a média pode ser afetado por outliers
    df["MonthlyIncome"] = df["MonthlyIncome"].fillna(df["MonthlyIncome"].median())

# NumberOfDependents tem poucos nulos ~2, então vou assumir que são 0, pois nesse caso é muito factivel e não acho que terá grande impacto no resultado

    df["NumberOfDependents"] = df["NumberOfDependents"].fillna(0)

# Agora irei tratar outliers nas colunas de atraso

# Colunas de atraso podem ter valores extremos (ex.: 98), esses valores podem criar falsas premissas no scorecard.
# Aqui irei capar em no máximo em 10: qualquer valor > 10 vira 10, assim será mantido a ideia de "Muito alto".
# Em risco de crédito, a informação útil costuma ser  em faixas de atraso: 0 (bom), 1–2 (alerta), 3+ (ruim) e muito alto (péssimo).

    df.loc[df["NumberOfTimes90DaysLate"] > 10, "NumberOfTimes90DaysLate"] = 10

# Por segurança, irei capa o modelo
# Abaixo o valores o percetil > que 99%, serão alterados para o percentil e 99%
# Isso evitar distorções no modelo, e mantém uma ideia parecida
    def cap_series(s: pd.Series, p_high=0.99):
        hi = s.quantile(p_high)
        return s.clip(upper=hi)

    df["RevolvingUtilizationOfUnsecuredLines"] = cap_series(df["RevolvingUtilizationOfUnsecuredLines"], p_high=0.99)
    df["DebtRatio"] = cap_series(df["DebtRatio"], p_high=0.99)
    df["MonthlyIncome"] = cap_series(df["MonthlyIncome"], p_high=0.99)
    df["NumberOfOpenCreditLinesAndLoans"] = cap_series(df["NumberOfOpenCreditLinesAndLoans"], p_high=0.99)
    df["NumberRealEstateLoansOrLines"] = cap_series(df["NumberRealEstateLoansOrLines"], p_high=0.99)
    df["NumberOfDependents"] = cap_series(df["NumberOfDependents"], p_high=0.99)

# Sanity Checker
    print("Shape final:", df.shape)
    print("Target rate:", df[COLUMN_TARGET].mean())
    print("Missing restantes (top 10):")
    print(df.isna().sum().sort_values(ascending=False).head(10))

# Salva

    df.to_csv("../data/train_clean.csv", index=False)
    print("Salvo: train_clean.csv")

if __name__ == "__main__":
    main()
"""
Output:

Shape final: (149999, 13)
Target rate: 0.06684044560297069
Missing restantes (top 10):
SeriousDlqin2yrs                        0
RevolvingUtilizationOfUnsecuredLines    0
age                                     0
NumberOfTime30-59DaysPastDueNotWorse    0
DebtRatio                               0
MonthlyIncome                           0
NumberOfOpenCreditLinesAndLoans         0
NumberOfTimes90DaysLate                 0
NumberRealEstateLoansOrLines            0
NumberOfTime60-89DaysPastDueNotWorse    0
dtype: int64

Salvo: train_clean.csv 
"""

Shape final: (149999, 13)
Target rate: 0.06684044560297069
Missing restantes (top 10):
SeriousDlqin2yrs                        0
RevolvingUtilizationOfUnsecuredLines    0
age                                     0
NumberOfTime30-59DaysPastDueNotWorse    0
DebtRatio                               0
MonthlyIncome                           0
NumberOfOpenCreditLinesAndLoans         0
NumberOfTimes90DaysLate                 0
NumberRealEstateLoansOrLines            0
NumberOfTime60-89DaysPastDueNotWorse    0
dtype: int64
Salvo: train_clean.csv


'\nOutput:\n\nShape final: (149999, 13)\nTarget rate: 0.06684044560297069\nMissing restantes (top 10):\nSeriousDlqin2yrs                        0\nRevolvingUtilizationOfUnsecuredLines    0\nage                                     0\nNumberOfTime30-59DaysPastDueNotWorse    0\nDebtRatio                               0\nMonthlyIncome                           0\nNumberOfOpenCreditLinesAndLoans         0\nNumberOfTimes90DaysLate                 0\nNumberRealEstateLoansOrLines            0\nNumberOfTime60-89DaysPastDueNotWorse    0\ndtype: int64\n\nSalvo: train_clean.csv \n'